<a href="https://colab.research.google.com/github/wdconinc/practical-computing-for-scientists/blob/master/Lectures/lecture12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lecture #12

##Standard Preamble

In [0]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math

##In our last episode

* Finding roots of functions of single variable
* Optimization
  * Connection between root finding and 1D optimization.
  * Finding minima and maxima of functions.


## Warm-up exercise

In [0]:
import string

message = "Yrg'f unir gur cebwrpg qhr ba Zbaqnl vafgrnq."

abc = "abcdefghijklmnopqrstuvwxyzYZ"
abc13 = abc[13:26] + abc[:13] + "LM"

rot13 = str.maketrans(abc, abc13)
str.translate(message, rot13)

## Optimization

"Optimization" refers to the process in which we minimimze or maximize a quantity. Finding the root of the derivative.

Exercise:

Find the minima and maxima of our test function

$$ f(x) = \cos 8x + x^2 -x $$

In [0]:
g = lambda x : np.cos(8*x) + x**2 - x
dg = lambda x : -8*np.sin(8*x) + 2*x - 1
x = np.linspace(-1,1.5,100)
plt.plot(x,g(x))

### Local or global minima within a range

<img src="https://upload.wikimedia.org/wikipedia/commons/6/68/Extrema_example_original.svg">

###Bracketing minima

In [0]:
def bracket_minima(f, a, b, ndivisions = 100):
    ''' 
    return a list of tuples (xlow, xmid, xhigh) that bracket each minimum
    '''
    result = []
    dx = (b - a) / ndivisions
    xfirst = a
    for i in range(ndivisions*2):
        xmid = xfirst + dx / 2.0
        xlast = xmid + dx / 2.0
        if f(xfirst) > f(xmid) and f(xlast) > f(xmid):
            # we have a minimum
            result.append((xfirst, xmid, xlast))
        xfirst = xmid
    return result

bracket_minima(g, -1.0, 1.5)

### Improving knowledge of the minima: golden ratio search

<img src="https://upload.wikimedia.org/wikipedia/commons/5/52/GoldenSectionSearch.png" width="400">

In [0]:
def golden_search(f, x1, x3, accuracy = 1e-6, nmax = 100):
    tau = (math.sqrt(5) - 1) / 2.0
    x2 = x1 + (x3 - x1) * (1 - tau)
    x4 = x1 + (x3 - x1) * tau
    f1, f2, f4, f3 = f(x1), f(x2), f(x4), f(x3)
    for i in range(nmax):
        print("iteration %i" % i)
        print("x1=%f, x2=%f, x4=%f, x3=%f" % (x1,x2,x4,x3))
        print("f1=%f, f2=%f, f4=%f, f3=%f" % (f1,f2,f4,f3))

        if f2 < f4: # f4a
            x3, f3, x4, f4 = x4, f4, x2, f2
            x2 = x1 + (x3 - x1) * (1 - tau)
            f2 = f(x2)
        else:       # f4b
            x1, f1, x2, f2 = x2, f2, x4, f4
            x4 = x1 + (x3 - x1) * tau
            f4 = f(x4)

        if math.fabs(x4 - x1) < accuracy: return x4

f = lambda x : (x - 0.25)**2

golden_search(f, 0.0, 1.0)

### Newton's method for optimization

As you know, from calculus, we can expand any well behaved function $f(x)$ in a series around a point $\bar{x}$:

$$f(x) = f(\bar{x}) + \frac{\partial f}{\partial x}\big|_\bar{x}(x-\bar{x}) + \frac{\partial^2 f}{\partial x^2}\big|_\bar{x} \frac{1}{2}(x-\bar{x})^2 $$

or with $\Delta x = x - \bar{x}$


$$f(x) = f(\bar{x} + \Delta x) = f(\bar{x}) + \frac{\partial f}{\partial x}\big|_\bar{x} \Delta x + \frac{\partial^2 f}{\partial x^2}\big|_\bar{x} \frac{1}{2} \Delta x^2 $$

We will successively improve our estimate of the minimum by choosing different points $\bar{x}_n$ that are closer to the minimum. We want $\bar{x}_n$ to be a stationary point, so that
$$ \frac{df(\bar{x} + \Delta x)}{d\Delta x} = 0$$

This happens when 
$$ \frac{\partial f}{\partial x}\big|_{\bar{x}_n} + \frac{\partial^2 f}{\partial x^2}\big|_{\bar{x}_n} \Delta x = 0 $$

So we update successively with this $\Delta x$:
$$ \bar{x}_{n+1} = \bar{x}_n + \Delta x =\bar{x}_n - \frac{\partial f}{\partial x}\big|_{\bar{x}_n} / \frac{\partial^2 f}{\partial x^2}\big|_{\bar{x}_n} $$


In [0]:
f = lambda x : (x - 0.25)**2

def D(f, x, h = 1e-6):
    temp = x+h
    h = temp-x
    return (f(x+h) - f(x-h)) / (2*h)

def DD(f, x, h = 1e-6):
    temp = x+h
    h = temp-x
    return (f(x+h) + f(x-h) - 2*f(x)) / h**2


def newton_optimize(f, xlow, xhigh, df = None, ddf = None, accuracy = 1e-6, nmax = 100):
    xbar = xlow
    fbar = f(xlow)
    fhigh = f(xhigh)
    if fhigh < fbar: 
        xbar, fbar = xhigh,fhigh

    # if user doesn't supply 1st and 2nd derivatives
    # make function objects to evaluate them
    if df == None:
        df = lambda x : D(f,x)
    if ddf == None:
        ddf = lambda x : DD(f,x)

    for i in range(nmax):
        x = xbar - df(xbar) / ddf(xbar)
        print(i, xbar, x, df(xbar), ddf(xbar))
        if math.fabs(x - xbar) < accuracy:
            return x, f(x)
        else:
            xbar = x
    raise ArithmeticError("Failed to converge")

In [0]:
print(newton_optimize(f, -1.0, +1.0))

##Multiple dimensions

####A test function: $f(\vec{x})=(x-1)^2 + (y-2)^2$ 

In [0]:
f = lambda x,y : (x - 1.0)**2 + (y - 2.0)**2

In [0]:
print("f(1.0, 1.0) = %f" % f(1.0, 1.0))
print("f(1.0, 2.0) = %f" % f(1.0, 2.0))

###How to plot a 2D scalar function

In [0]:
xrange = np.linspace(-4.0, +4.0, 100)
yrange = np.linspace(-4.0, +4.0, 100)
xpts, ypts = np.meshgrid(xrange, yrange)

In [0]:
print(xrange)

In [0]:
print(xpts)
print(ypts)

In [0]:
for x,y in zip(xpts,ypts):
  print(f(x,y))

In [0]:
[f(x,y) for x,y in zip(xpts,ypts)]

In [0]:
fpts = np.array([f(x,y) for x,y in zip(xpts,ypts)])

In [0]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.plot_surface(xpts, ypts, fpts)

In [0]:
plt.contour(xpts, ypts, fpts)

In [0]:
plt.contourf(xpts, ypts, fpts)
plt.plot([1],[2], "-or")

###How to find the minimum?

As you know, from calculus, we can expand any well behaved function $f(\vec{x})$ in a series around a point $\vec{x}$:

$$f(\vec{x} + \Delta\vec{x}) = f(\vec{x}) + \nabla f(\vec{x})\big|_\vec{x} \Delta\vec{x} + \frac{1}{2} H f(\vec{x})\big|_\vec{x} \Delta\vec{x}^2 $$

We will successively improve our estimate of the minimum by choosing different points $\bar{x}_n$ that are closer to the minimum. We want $\bar{x}_n$ to be a stationary point, so that
$$ \frac{df(\vec{x} + \Delta\vec{x})}{d\Delta\vec{x}} = 0$$

This happens when 
$$ \nabla f\big|_{\vec{x}} + H f\big|_{\vec{x}} \Delta \vec{x} = 0 $$

So we update successively with this $\Delta\vec{x}$:
$$ \vec{x}_{n+1} = \vec{x}_n + \Delta \vec{x} =\vec{x}_n - \big[ H f \big|_{\vec{x}_n} \big]^{-1} \nabla f\big|_{\vec{x}_n} $$


### The Newton method for functions of more than one variable

###The partial derivative

Consider the function $f(x_0,x_1,x_2) = 2 x_0 + 3 x_1 + 5 x_1 x_2$

It is easier to consider the arguments $x_i$ to be part of a single vector argument $\vec{x}$

In [0]:
def partial(f, i, h = 1e-6):
    ''' 
        Returns a function object to compute the partial derivative of f with respect to x[i].
        
        f(x) is assumed to be a scalar function of a vector or scalar argument x.
    '''
    def df(x, f = f, i = i, h = h):
        x = np.array(x, dtype = np.float64) # make a copy and assure the use of 64-bit floats
        x[i] += h
        f_plus = f(x)
        x[i] -= 2*h
        f_minus = f(x)
        return (f_plus - f_minus) / (2.0*h)
    # note, partial() returns a function object, not the result of the function
    return df

fmulti = lambda x: 2*x[0] + 3*x[1] + 5*x[1]*x[2]

In [0]:
df0 = partial(fmulti,2)
v = [1,1,1]
print(df0(v))

In [0]:
df1 = partial(fmulti,1)
df2 = partial(fmulti,2)
print(df1(v),df2(v))

In [0]:
df12 = partial(df1,2)
print(df12(v))

###A little bit about matrices

We now need to represent vector quantities such as $\nabla f(\vec{x})$ and the Hessian matrix $H$. This means using matrices and matrix algebra.  Here's a quick introduction:

In [0]:
import numpy.matlib as ml

# make a 2x2 (row x column) matrix
M = ml.matrix([[1,2],[3,4]])
print(M)
print(type(M))

In [0]:
# make a 1x2 "matrix" (a.k.a. a row vector)
V = ml.matrix([1,2])
print(V)
print(V.T) # the transpose of V = a 2x1 matrix = a column vector

In [0]:
print(M.dot(V.T)) # the dot product
print(M.I) # the matrix inverse
print(M.I * M) # checking the inverse. * is a other way to take the dot product

###The gradient

In [0]:
def gradient(f, x, h = 1e-6):
    ''' return the gradient of f(x) as a column vector with length = len(x) '''
    v = ml.matrix([partial(f, i, h = h)(x) for i in range(len(x))], dtype = np.float64)
    return v.T

print(gradient(fmulti,v))

###The Hessian matrix

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/ceb2ef7133d4ffb011021db5f90126d42058378d">

In [0]:
def hessian(f, x, h = 1e-6):
    ''' returns the two dimensional matrix of second partial derivatives of f(x). a.k.a. the Hessian matrix '''
    v = [ [ partial(partial(f,column),row)(x) for column in range(len(x)) ] for row in range(len(x))]
    mx = ml.matrix(v, dtype = np.float64)
    return ml.matrix(v,dtype = np.float64)

####A test function: $f(\vec{x})=(x-1)^2 + (y-2)^2$ 

Using obscure `numpy.mgrid` tricks: https://docs.scipy.org/doc/numpy/reference/generated/numpy.mgrid.html

In [0]:
f = lambda x : (x[0]-1)**2 + (x[1]-2)**2
x, y = np.mgrid[-2.5:3:101j, 0:4:101j]

####The `multi_newton` function

In [0]:
def multi_newton(f, xguess, h = 1e-6, accuracy = 1e-6, nmax=100, want_points = False, debug = False):
    xbar = ml.matrix(xguess, dtype = np.float64).T # to get a column vector
    xpoints = [xbar.A1[0]]
    ypoints = [xbar.A1[1]]
    for i in range(nmax):
        H = hessian(f, xbar.A1)
        grad = gradient(f, xbar.A1) # to get a column vector
        if debug:
            print("iteration ", i)
            print("xbar", xbar)
            print("grad=", grad)
            print("H=", H)
            print("H.I=", H.I)
            print("H.I.dot(grad)", H.I.dot(grad))
        x = xbar - H.I.dot(grad)
        if debug: print("x=", x)
        xpoints.append(x.A1[0])
        ypoints.append(x.A1[1])
        if np.sum((x.A1 - xbar.A1)**2) < accuracy**2:
            if not want_points:
                return x.A1
            else:
                return x.A1, xpoints, ypoints
        else:
            xbar = x
    raise ArithmeticError("Failed to converge")

####A somewhat more difficult test function

In [0]:
fharder = lambda x : 1 - x[0]*np.exp(-x[0]) + (x[1] - 2)**2

####Testing `multi_newton` on the new test function

In [0]:
minx, xpoints, ypoints = multi_newton(fharder, [1.8,2.5], want_points = True)
cs = plt.contourf(X, Y, fharder([X,Y]), 100)
plt.colorbar(cs)
print(xpoints, ypoints)
plt.plot(xpoints, ypoints, "-ow")